LDA without using built in Python libary



In [16]:
import random


In [17]:
def transpose(M):
    return list(map(list, zip(*M)))

def matmul(A, B):
    # Multiply A (m×n) by B (n×p)
    result = [[0]*len(B[0]) for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def subtract_vec(a, b):
    return [ai - bi for ai, bi in zip(a, b)]

def outer(a, b):
    return [[ai * bj for bj in b] for ai in a]

def mean_vec(X):
    n = len(X)
    d = len(X[0])
    return [sum(row[i] for row in X) / n for i in range(d)]

def inverse_2x2(M):
    det = M[0][0]*M[1][1] - M[0][1]*M[1][0]
    return [[ M[1][1]/det, -M[0][1]/det],
            [-M[1][0]/det,  M[0][0]/det]]


In [18]:
def lda_2class(X, y):
    # Split by class
    X1 = [X[i] for i in range(len(X)) if y[i] == 0]
    X2 = [X[i] for i in range(len(X)) if y[i] == 1]

    # Class means
    m1, m2 = mean_vec(X1), mean_vec(X2)
    overall_mean = mean_vec(X)

    # Within-class scatter Sw
    Sw = [[0,0],[0,0]]
    for row in X1:
        diff = subtract_vec(row, m1)
        Sw = [[Sw[i][j] + outer(diff, diff)[i][j] for j in range(2)] for i in range(2)]
    for row in X2:
        diff = subtract_vec(row, m2)
        Sw = [[Sw[i][j] + outer(diff, diff)[i][j] for j in range(2)] for i in range(2)]

    # Between-class scatter Sb
    mean_diff = subtract_vec(m1, m2)
    Sb = outer(mean_diff, mean_diff)

    # Solve eigenvector of inv(Sw)*Sb
    Sw_inv = inverse_2x2(Sw)
    M = matmul(Sw_inv, Sb)

    # For 2x2, eigenvector can be approximated as the column with larger variance
    w = [M[0][0] - M[1][1], M[0][1] + M[1][0]]

    # Normalize w
    norm = sum(val**2 for val in w) ** 0.5
    w = [val/norm for val in w]

    # Project data
    projected = [sum(xi*wi for xi, wi in zip(row, w)) for row in X]

    return w, projected

In [19]:
random.seed(42)
X = [[random.random(), random.random()] for _ in range(20)]
y = [0]*10 + [1]*10   # first 10 belong to class 0, next 10 to class 1

w, proj = lda_2class(X, y)

print("Projection vector w:", w)
print("First 5 projected values:", proj[:5])

Projection vector w: [-0.5598788660512152, 0.8285744718184392]
First 5 projected values: [-0.33727827754495737, 0.030963616566523605, 0.14836125210573836, -0.427477187354222, -0.2115358946100972]
